In [18]:
import csv

def load_csv(csv_file):
    with open(csv_file, 'r') as csv_file:
        csv_reader = csv.reader(csv_file)
        stores = list(csv_reader)
        headers = stores[0]
        return [{i: j for i, j in zip(headers, store)}for store in stores[1:]]

stores = load_csv('stores_table.csv')
zip_data = load_csv('zip_code_data_rows.csv')
zip_demo = load_csv('zip_code_demographics_rows.csv')
zip_data2 = load_csv('zip_codes_rows.csv')
zip_household = load_csv('calculated_household_income_table_rows.csv')

In [2]:
import math

def haversine(lat1, lon1, lat2, lon2):
    # Radius of the Earth in kilometers
    R = 6371.0

    # Convert degrees to radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # Differences in coordinates
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    # Haversine formula
    a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c

    return distance


In [19]:
zip_codes = {}

for data in zip_data + zip_demo + zip_data2 + zip_household:
    zipc = data['zip_code']
    if zipc not in zip_codes:
        zip_codes[zipc] = {}
    for k, v in data.items():
        if k != 'zip_code':
            zip_codes[data['zip_code']][k] = v

In [4]:
stores[0], zip_codes['5009']

({'DATE': '2021-07-01',
  'PARENT_NAME': "McDonald's",
  'COMPANY_NAME': "McDonald's US",
  'STORE_NAME': '',
  'STORE_ID': '-1.00035E+18',
  'STREET': '6005 Liberty Rd',
  'CITY': 'Baltimore',
  'STATE': 'Maryland',
  'COUNTRY': 'United States',
  'LATITUDE': '39.3367215390362',
  'LONGITUDE': '-76.71288773',
  'CENSUS_BG_KEY': '2.40054E+11',
  'case_data_sid': '267147',
  'location': '0101000020E61000007684DBF39F2D53C07A02FFB019AB4340'},
 {'GEO_ID': '860Z200US05009',
  'household_total': '0',
  'median_household_income': '-',
  'mean_household_income': '-',
  'total_families': '0',
  'median_family_income': '-',
  'mean_family_income': '-',
  'median_age': '-',
  'sex_ratio': '-',
  'age_dependency_ratio': '-',
  'undergraduate_students': '0',
  'employment_population_ratio': '-',
  'total_workers': '0',
  'workers_not_from_home': '0',
  'commute_vehicles': '-',
  'median_household_value': '-',
  'lat': '43.64769',
  'lng': '-72.34395',
  'city': 'White River Junction',
  'state_id':

In [5]:
stores_by_state = {}

for store in stores:
    state = store['STATE']
    if state not in stores_by_state:
        stores_by_state[state] = []
    stores_by_state[state].append(store)

In [6]:
store_names = {
    "McDonald's": 0,
    "Starbucks": 1,
    "Dunkin'": 2
}

In [7]:
from tqdm import tqdm
store_by_zip_count = {}

for z, data in tqdm(zip_codes.items()):
    if 'radius' not in data or not data['radius']:
        continue

    lat_1 = float(data['lat'])
    lng_1 = float(data['lng'])
    for store in stores_by_state[data['state_name']]:
        lat_2 = float(store['LATITUDE'])
        lng_2 = float(store['LONGITUDE'])
        distance = haversine(lat_1, lng_1, lat_2, lng_2)
        radius = float(data['radius']) * 5
        if distance <= radius:
            if z not in store_by_zip_count:
                store_by_zip_count[z] = [0, 0, 0]
            
            store_by_zip_count[z][store_names[store['PARENT_NAME']]] += 1
        


  0%|          | 44/33791 [00:00<04:51, 115.77it/s]

100%|██████████| 33791/33791 [02:23<00:00, 235.67it/s] 


In [9]:
store_by_zip_count['7832']

[6, 8, 14]

In [20]:

for z, (a, b, c) in store_by_zip_count.items():
    zip_codes[z]['McDonald\'s'] = a
    zip_codes[z]['Starbucks'] = b
    zip_codes[z]['Dunkin\''] = c
zip_codes['7832']

{'GEO_ID': '860Z200US07832',
 'household_total': '1209',
 'median_household_income': '104205',
 'mean_household_income': '135013',
 'total_families': '989',
 'median_family_income': '112762',
 'mean_family_income': '141227',
 'median_age': '49.5',
 'sex_ratio': '98.4',
 'age_dependency_ratio': '62.3',
 'undergraduate_students': '116',
 'employment_population_ratio': '59.2',
 'total_workers': '1403',
 'workers_not_from_home': '1269',
 'commute_vehicles': '1180',
 'median_household_value': '297000',
 'lat': '40.94755',
 'lng': '-75.06385',
 'city': 'Columbia',
 'state_id': 'NJ',
 'state_name': 'New Jersey',
 'zcta': 'true',
 'parent_zcta': '',
 'population': '3123',
 'density': '28.7',
 'county_fips': '34041',
 'county_name': 'Warren',
 'county_weights': '{"34041": 100}',
 'county_names_all': 'Warren',
 'county_fips_all': '34041',
 'imprecise': 'false',
 'military': 'false',
 'timezone': 'America/New_York',
 'area': '108.815331010453',
 'location': '0101000020E610000060764F1E16C452C093A9

In [21]:
output = []

headers = ['zip_code',
           'median_age',
 'sex_ratio',
 'age_dependency_ratio',
 'undergraduate_students',
 'employment_population_ratio',
 'total_workers',
 'workers_not_from_home',
 'commute_vehicles',
 'median_household_value',
 'state_name',
 'population',
 'density',
 'county_fips',
 'county_name',
 'area',
 'radius',
 'calculated_household_total',
 'McDonald\'s',
 'Starbucks',
 'Dunkin\'']

output.append(headers)

for z, data in zip_codes.items():
    row = [z]
    for h in headers[1:]:
        if h in data:
            row.append(data[h])
        else:
            row.append('')
    output.append(row)
output

[['5009',
  '-',
  '-',
  '-',
  '0',
  '-',
  '0',
  '0',
  '-',
  '-',
  'Vermont',
  '0',
  '0',
  '50027',
  'Windsor',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['10519',
  '34.1',
  '647.6',
  '96.3',
  '0',
  '66.7',
  '80',
  '80',
  'null',
  '-',
  'New York',
  '157',
  '242.6',
  '36119',
  'Westchester',
  '0.647155812036274',
  '0.453867924480729',
  '',
  '',
  '',
  ''],
 ['11794',
  '19.5',
  '90.9',
  '3.1',
  '1230',
  '31.9',
  '374',
  '349',
  '60',
  '-',
  'New York',
  '1279',
  '2161.3',
  '36103',
  'Suffolk',
  '0.591773469671031',
  '0.434013070975487',
  '',
  0,
  4,
  1],
 ['14260',
  '19',
  '103',
  '3.2',
  '797',
  '24.8',
  '194',
  '181',
  '75',
  '-',
  'New York',
  '814',
  '1270.1',
  '36029',
  'Erie',
  '0.640894417762381',
  '0.451666944964728',
  '',
  0,
  4,
  0],
 ['15539',
  '32.6',
  '48.9',
  '52.2',
  '0',
  '65.4',
  '138',
  '138',
  '140',
  '118600',
  'Pennsylvania',
  '280',
  '90.2',
  '42009',
  'Bedford',
  '3.10421286031042',


In [14]:
"median_age,sex_ratio,age_dependency_ratio,undergraduate_students,employment_population_ratio,total_workers,workers_not_from_home,commute_vehicles,median_household_value".split(',')

['median_age',
 'sex_ratio',
 'age_dependency_ratio',
 'undergraduate_students',
 'employment_population_ratio',
 'total_workers',
 'workers_not_from_home',
 'commute_vehicles',
 'median_household_value']

In [10]:
import json
with open('store_by_zip_count.json', 'w') as f:
    json.dump(store_by_zip_count, f)